# BLAST 예제

이 예제는 BLAST를 이해하기 위해 코드레벨에서 실행을 해보는 예제이다.
임의의 염기서열이 들어왔을때, BLAST를 통해서 어떻게 정렬이 진행되는지 확인하고자한다.

In [1]:
!pip install biopython

## Query서열 만들기

임의의 염기서열(Query)를 확보하기 위해서, 우선 NCBI에서 제공하는 누룩곰팡이 서열을 검색하여 일부를 발췌한다.
아래와 같이 확인할 수 있으며, 단락당 길이 70으로 구분되어있으므로 이부분을 정제해서 사용하도록 한다.


>WHVT01000001.1 Aspergillus fischeri strain NRRL4161 NODE_1_length_1856684_cov_136.405182, whole genome shotgun sequence
CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA
CCCTAACTTTGAGAAAGATGTGAGGTTTTATATTGTCTAGTTAGCACACTACCCTATTGTGCTAGGAATA
CCCTGGATAAAGGTACATAACCCCCTGATATAATTCACAGCCCACAGTATTATATTTAATAGTGATTACT

In [1]:
nucleotide_seq = "CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAA CCCTAACTTTGAGAAAGATGTGAGGTTTTATATTGTCTAGTTAGCACACTACCCTATTGTGCTAGGAATA CCCTGGATAAAGGTACATAACCCCCTGATATAATTCACAGCCCACAGTATTATATTTAATAGTGATTACT"
nucleotide_seq = nucleotide_seq.replace(" ", "")
nucleotide_seq

'CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACTTTGAGAAAGATGTGAGGTTTTATATTGTCTAGTTAGCACACTACCCTATTGTGCTAGGAATACCCTGGATAAAGGTACATAACCCCCTGATATAATTCACAGCCCACAGTATTATATTTAATAGTGATTACT'

## NCBI에서 제공하는 BLAST API 이용하기

NCBIWWW 모듈에서 qblast 함수를 제공한다. NCBI 서버에서 제공하는 BLAST를 수행하게되며, return은 xml 형태이다.
Sequence 길이에 따라 상이하겠지만, 상당히 오랜 시간이 걸린다. Database에 있는 모든 Sequence와 query를 정렬하기 때문

In [9]:
import time
from Bio.Blast import NCBIWWW


st = time.time()
res = NCBIWWW.qblast(program='blastn', database='nt', sequence=nucleotide_seq)
ed = time.time()
print(f'총 걸린시간: {ed-st}')

총 걸린시간: 731.6572370529175


BLAST를 실행하는데 너무 오래걸리므로 해당 파일은 저장한 후, 계속 사용할 수 있도록 한다.

In [10]:
with open('../sample_data/ex_blast.xml', 'w', encoding='utf-8') as f:
    f.write(res.read())

## BLAST 결과 확인

In [2]:
from Bio.Blast import NCBIXML

with open('../sample_data/ex_blast.xml', 'r', encoding='utf-8') as f:
    blast_results = NCBIXML.parse(f)
    for blast_result in blast_results:
        for alignment in blast_result.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < 0.0001:
                    print('****Alignment****')
                    print('sequence: ', alignment.title)
                    print('length: ', alignment.length)
                    print('e value: ', hsp.expect)
                    print(hsp.query[0:75] + '...')
                    print(hsp.match[0:75] + '...')
                    print(hsp.sbjct[0:75] + '...')

****Alignment****
sequence:  gi|11991765|gb|AF202956.1| Aspergillus fumigatus long terminal repeat, complete sequence; pol gene, partial sequence; and long terminal repeat, complete sequence
length:  5820
e value:  4.09826e-33
ACTTTGAGAAAGATGTGAGGTTTTATATTGTCTAGTTAGCACACTACCCTATTGTGCTAGGAATACCCTGGATAA...
||||| |||||||| | |||||||||||| ||||||||||||| |||| |||| | ||||||||||| | |||||...
ACTTTAAGAAAGATATAAGGTTTTATATTATCTAGTTAGCACATTACCTTATTATACTAGGAATACCTTAGATAA...
****Alignment****
sequence:  gi|2063804863|emb|OU343212.1| Thunnus maccoyii genome assembly, chromosome: 22
length:  26962447
e value:  2.42229e-23
CTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTA...
|||||| ||||||||||||||||| ||||||||||| ||||| ||||||||||||||||||||||||||||||||...
CTAACCATAACCCTAACCCTAACCATAACCCTAACCATAACCATAACCCTAACCCTAACCCTAACCCTAACCCTA...
****Alignment****
sequence:  gi|1795661398|emb|LR738421.1| Lutra lutra genome assembly, chromosome: X
length:  99689450
e value:  1.34091e-07
CCTAACCCT

함수들에 대해서 하나씩 살펴보면,
- `alignment.title`에는 DB에서 조회된 염기서열의 메타정보가 들어있다.
- `alignment.length`에는 DB에서 조회된 염기서열의 전체 길이를 나타낸다.
- `hsp.expect`에는 hsp(high score pair)의 e_value가 들어있다. **e-value가 낮을수록 신뢰할만한 수치**라고 판단한다.
- `hsp.query`에는 내가 입력으로 넣은 염기서열이 들어온다. BLAST를 통해 정렬이 된 영역만을 보여준다.
- `hsp.sbjct`에는 DB의 염기서열이 들어온다. 마찬가지로 정렬이 된 영역만을 보여준다.
- `hsp.match`에는 실제로 match가 된 영역을 보여준다. `|`로 표현된것은 Exact Match가 된 부분을 나타내고, 공백으로 된 부분은 Match가 되지 않은 영역을 보여준다.

그럼 결과에 대해서 해석을 본다.
1. 가장 낮은 e_value를 가지고 있는, 길이 5,820짜리의 염기서열은 "누룩곰팡이의 긴 반복서열" 이라고 한다.
2. 두번째로 낮은 e_value를 가지고 있는, 길이 26,962,447짜리의 염기서열은 "남방참다랑어의 22번째 염색체" 라고 한다.
3. 세번째로 낮은 e_value를 가지고있는, 길이 99,689,450짜리의 염기서열은 "수달의 X염색체" 라고 한다.

이처럼 Query서열을 통해서 Reference서열과의 정렬을 통해 Query서열이 어떠한 "종"인지 또는 "염색체"인지 또는 "유전자"인지 등을 식별할 수 있게 된다.
물론 여기에서의 문제는 Query서열이 "누룩곰팡이의 긴 반복서열"인지, "남방참다랑어의 22번째 염색체"인지, "수달의 X염색체"인지는 아직 알 수 없지만, 후보군을 뽑았다는것에는 상당한 의미가 있다. 이 예제에서는 누룩곰팡이 서열의 일부를 사용했으므로, 우리는 Query로 발췌한 일부서열이 "누룩곰팡이의 긴 반복서열"임을 특정할 수 있다.

하지만, 실제로는 Query 서열이 어떠한 서열인지 알지 못하는 상태이기 때문에, 후보군을 좀더 분석해야 정확한 식별이 가능하다.